In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from functools import lru_cache
import re

In [14]:
df_statements = pd.read_csv("data/annual_statements.csv")
df_prices = pd.read_csv("data/monthly_returns.csv")

In [3]:
df_statements['datadate'] = pd.to_datetime(df_statements['datadate'])

In [4]:
df_statements.head()

,Unnamed: 0,gvkey,tic,fyear,datadate,conm,revt,ni,emp,prcc_f,ceq,oancf,at,dltt,act,lct,csho,cogs
0,0,1004,AIR,2002.0,2003-05-31,AAR CORP,606.337,-12.410,2.1,4.50,294.988,34.733,686.621,164.658,396.412,203.575,31.851,496.747
1,1,1004,AIR,2003.0,2004-05-31,AAR CORP,651.958,3.504,2.3,9.58,301.684,14.572,709.292,248.666,432.204,131.261,32.245,523.302
2,2,1004,AIR,2004.0,2005-05-31,AAR CORP,747.848,15.453,2.6,16.04,314.744,50.938,732.230,227.159,474.542,160.025,32.586,598.172
3,3,1004,AIR,2005.0,2006-05-31,AAR CORP,897.284,35.163,3.3,24.08,422.717,-40.482,978.819,318.576,624.454,187.788,36.654,704.081
4,4,1004,AIR,2006.0,2007-05-31,AAR CORP,1061.169,58.660,3.9,32.50,494.243,-21.239,1067.633,253.611,645.721,256.506,37.729,837.171


In [17]:
df_prices.head()

,Unnamed: 0,AIR,ADCT.1,AAL,ASA,PNW,PRG,ABT,SERV.1,WDDD,...,APG,NVT,CSTPF,ACA,CTRM,BBUS,IMUX,ARMP,SNOW,HYFM
0,2003-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2003-02,-0.065126,NaN,NaN,-0.094070,-0.006187,-0.085308,-0.060030,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
2,2003-03,-0.150562,NaN,NaN,-0.037873,0.088408,0.044042,0.055868,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.586207,NaN,NaN
3,2003-04,0.026455,NaN,NaN,0.015850,-0.000601,0.049628,0.080297,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.586957,NaN,NaN
4,2003-05,0.159794,NaN,NaN,0.018440,0.154016,0.026004,0.103219,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.643836,NaN,NaN


In [20]:
def build_decile_memberships(df_statements: pd.DataFrame,
                             id_col: str = 'tic',
                             n_bins: int = 10):
    """Return nested dict: deciles_by_year[year][decile] -> list of tickers."""
    df = df_statements.copy()

    # --- Basic sanity ---
    # Ensure expected columns exist
    req = {'gvkey','tic','fyear','datadate','revt','ni','oancf','ceq','prcc_f','csho'}
    missing = req - set(df.columns)
    if missing:
        raise ValueError(f"df_statements missing columns: {missing}")

    # Parse date + sort
    # make sure dates are sorted within gvkey
    df['datadate'] = pd.to_datetime(df['datadate'])
    df = df.sort_values(['gvkey','datadate']).reset_index(drop=True)

    # --- Market equity & core ratios (at fiscal-year end) ---
    df['ME'] = df['prcc_f'] * df['csho']   # assumes price $/share and shares in millions
    # Guard against divide-by-zero or negative ME
    df.loc[~(df['ME'] > 0), 'ME'] = np.nan

    df['bm'] = df['ceq']   / df['ME']              # Book-to-Market
    df['ep'] = df['ni']    / df['ME']              # Earnings-to-Price
    df['cp'] = df['oancf'] / df['ME']              # CashFlow-to-Price

    # --- GS: pre-formation 5y average sales growth ---
    # First compute YoY growth in revenue per firm
    df['revt_lag'] = df.groupby('gvkey')['revt'].shift(1)
    df['sales_growth'] = (df['revt'] / df['revt_lag']) - 1.0
    # Pre-formation average over *prior* 5 years (exclude current fyear reading)
    gs = (df.groupby('gvkey')['sales_growth']
        .apply(lambda s: s.shift(1).rolling(5, min_periods=5).mean())
        .reset_index(level=0, drop=True))   # <- key: drop the gvkey level

    df['gs'] = gs

    # --- Formation year (end of April Y uses fyear=Y-1 financials) ---
    df['formation_year'] = df['fyear'].astype('Int64') + 1

    # Keep the columns we need
    chars = df[['formation_year','gvkey','tic','bm','ep','cp','gs','ME']].dropna(subset=['formation_year'])

    # Helper to form deciles for one characteristic
    def one_char_deciles(chars_df: pd.DataFrame, char: str):
        cs = chars_df[['formation_year','gvkey','tic',char,'ME']].dropna(subset=[char]).copy()

        out_rows = []
        for Y, g in cs.groupby('formation_year'):
            # Need enough names to form deciles
            if g[char].nunique() < n_bins:
                continue
            # qcut on rank to avoid duplicate-bin issues
            g = g.copy()
            rk = g[char].rank(method='first')
            try:
                g['decile'] = pd.qcut(rk, n_bins, labels=False) + 1  # 1..10
            except ValueError:
                # fallback: skip if still not enough spread
                continue

            # Collect members per decile
            members = (g.groupby('decile')[id_col]
                         .apply(list)
                         .reindex(range(1, n_bins+1), fill_value=[])
                         .to_dict())
            out_rows.append((Y, members))

        # Build nested dict: deciles_by_year[year][decile] -> list
        deciles_by_year = {}
        for Y, members in out_rows:
            deciles_by_year[int(Y)] = {int(k): v for k, v in members.items()}
        return deciles_by_year

    return {
        'bm' : one_char_deciles(chars, 'bm'),
        'ep' : one_char_deciles(chars, 'ep'),
        'cp' : one_char_deciles(chars, 'cp'),
        'gs' : one_char_deciles(chars, 'gs'),
    }
    
deciles_by_year = build_decile_memberships(df_statements)

In [21]:
# example: show a couple stocks in the first decile in year 2020
# here, lower decile = low B/M ratio = glamour stock
deciles_by_year['bm'][2020][1][:5]

['AAL', 'WDDD', 'PBAJ', 'ANDR', 'BA']

In [38]:
def _as_period_m(df: pd.DataFrame) -> pd.DataFrame:
    """Ensure monthly PeriodIndex; tolerate string or Timestamp index."""
    if isinstance(df.index, pd.PeriodIndex) and df.index.freqstr == "M":
        return df
    try:
        idx = pd.PeriodIndex(df.index, freq="M")
    except Exception:
        idx = pd.to_datetime(df.index).to_period("M")
    out = df.copy()
    out.index = idx
    return out

def _compound_12m(window: pd.Series) -> float:
    """Compound 12 monthly decimal returns; require all 12 finite values."""
    if window.shape[0] != 12:
        return np.nan
    w = window.astype(float)
    if not np.isfinite(w).all():
        return np.nan
    return float((1.0 + w).prod() - 1.0)

def _nanmean_rowwise(arr: np.ndarray, axis=0):
    """nanmean without warnings: NaN if all-NaN."""
    if arr.size == 0:
        return np.full((arr.shape[1-axis],), np.nan, dtype=float)
    counts = np.sum(~np.isnan(arr), axis=axis, dtype=float)
    sums = np.nansum(arr, axis=axis)
    out = np.divide(sums, counts, out=np.full_like(sums, np.nan, dtype=float), where=counts>0)
    return out

def _nanprod1p_over_row(row: np.ndarray) -> float:
    """Compound across (1+r) ignoring NaNs; NaN if none."""
    valid = row[np.isfinite(row)]
    if valid.size == 0:
        return np.nan
    return float(np.prod(1.0 + valid) - 1.0)

# --- core: post-formation using df_prices monthly RETURNS (decimal) ---

def postformation_returns_5y_from_df(df_prices: pd.DataFrame,
                                     ticker: str,
                                     formation_year: int) -> list[float]:
    """
    df_prices: monthly RETURNS (decimal), index monthly, cols tickers.
    R_k windows: May(Y+k-1)..Apr(Y+k).
    """
    if ticker not in df_prices.columns:
        return [np.nan]*5

    s = _as_period_m(df_prices)[ticker]

    out = []
    for k in range(5):
        y_start = formation_year + k
        p0 = pd.Period(f"{y_start}-05", freq="M")
        months = [p0 + i for i in range(12)]
        try:
            window = s.loc[months]
        except KeyError:
            out.append(np.nan)
            continue
        out.append(_compound_12m(window))
    return out

# --- cached wrapper bound to df_prices ---

def make_pf5y_cached(df_prices: pd.DataFrame):
    dfp = _as_period_m(df_prices)

    @lru_cache(maxsize=None)
    def _pf5y_cached(ticker: str, formation_year: int):
        try:
            vals = postformation_returns_5y_from_df(dfp, ticker, formation_year)
            if vals is None or len(vals) != 5:
                return (np.nan,)*5
            return tuple(float(x) if x is not None else np.nan for x in vals)
        except Exception:
            return (np.nan,)*5
    _pf5y_cached.cache_clear()

    return _pf5y_cached

# --- cohort & panel ---

def _cohort_equal_weight_R1toR5(tickers, formation_year, _pf5y_cached_fn):
    if not tickers:
        return [np.nan]*5
    rets = np.array([_pf5y_cached_fn(t, formation_year) for t in tickers], dtype=float)  # [N x 5]
    out = _nanmean_rowwise(rets, axis=0)  # -> [5], NaN if cohort has no valid data
    return out.tolist()

def build_panel_for_char(deciles_by_year, df_prices, char_key='bm', deciles=10, years=None):
    by_year = deciles_by_year[char_key]
    Ys_all = sorted(by_year.keys())
    years = Ys_all if years is None else [y for y in years if y in by_year]

    _pf5y_cached_fn = make_pf5y_cached(df_prices)

    sum_R = np.zeros((5, deciles), dtype=float)
    cnt_R = np.zeros((5, deciles), dtype=float)

    for Y in years:
        for d in range(1, deciles+1):
            tickers = by_year.get(Y, {}).get(d, [])
            r1to5 = np.array(_cohort_equal_weight_R1toR5(tickers, Y, _pf5y_cached_fn), dtype=float)
            mask = np.isfinite(r1to5)
            sum_R[mask, d-1] += r1to5[mask]
            cnt_R[mask, d-1] += 1.0

    with np.errstate(invalid='ignore', divide='ignore'):
        panel = sum_R / np.where(cnt_R==0.0, np.nan, cnt_R)

    panel_df = pd.DataFrame(panel,
                            index=[f"R{i}" for i in range(1,6)],
                            columns=[str(d) for d in range(1, deciles+1)])

    ar = pd.Series(_nanmean_rowwise(panel, axis=0), index=panel_df.columns, name="AR")
    cr5 = pd.Series([_nanprod1p_over_row(panel[:, j]) for j in range(panel.shape[1])],
                    index=panel_df.columns, name="CR_5y")

    return panel_df, ar, cr5

In [46]:
# Load the monthly returns you already built
df_prices = pd.read_csv("data/monthly_returns4.csv", index_col=0)

# Build panel for, say, book-to-market deciles
panel_df, ar, cr5 = build_panel_for_char(deciles_by_year, df_prices, char_key='bm', deciles=10)

print(panel_df)  # rows R1..R5 by decile
print(ar)        # average annual return across R1..R5 per decile
print(cr5)       # 5-year compounded return per decile


           1         2         3         4         5         6         7  \
R1  0.230864  0.165150  0.154330  0.130206  0.152722  0.189832  0.141165   
R2  0.122684  0.109656  0.125627  0.149815  0.116547  0.110811  0.117326   
R3  0.120758  0.112367  0.149921  0.117748  0.121103  0.116944  0.125622   
R4  0.085786  0.111583  0.101683  0.110637  0.099287  0.108779  0.114245   
R5  0.119981  0.132974  0.113424  0.097341  0.110731  0.127159  0.110761   

           8         9        10  
R1  0.172715  0.202299  0.284169  
R2  0.132551  0.131110  0.214504  
R3  0.114867  0.147945  0.186675  
R4  0.133613  0.148052  0.163335  
R5  0.093090  0.127691  0.171839  
1     0.136015
2     0.126346
3     0.128997
4     0.121149
5     0.120078
6     0.130705
7     0.121824
8     0.129367
9     0.151419
10    0.204105
Name: AR, dtype: float64
1     0.883364
2     0.811259
3     0.832777
4     0.770284
5     0.761842
6     0.844962
7     0.776324
8     0.834823
9     1.021109
10    1.523051
Name: CR

In [47]:
# Load the monthly returns you already built
df_prices = pd.read_csv("data/monthly_returns4.csv", index_col=0)

# Build panel for, say, book-to-market deciles
panel_df, ar, cr5 = build_panel_for_char(deciles_by_year, df_prices, char_key='cp', deciles=10)

print(panel_df)  # rows R1..R5 by decile
print(ar)        # average annual return across R1..R5 per decile
print(cr5)       # 5-year compounded return per decile


           1         2         3         4         5         6         7  \
R1  0.227251  0.173466  0.160419  0.157552  0.139625  0.162757  0.146482   
R2  0.178292  0.085334  0.064805  0.136292  0.126109  0.128571  0.117178   
R3  0.111257  0.115339  0.051022  0.144934  0.123267  0.110023  0.136120   
R4  0.074448  0.080854  0.097834  0.100324  0.093644  0.101602  0.118173   
R5  0.117135  0.163243  0.107877  0.119048  0.107860  0.112476  0.127549   

           8         9        10  
R1  0.173720  0.181839  0.291467  
R2  0.131451  0.151315  0.207648  
R3  0.135601  0.137397  0.208813  
R4  0.124371  0.134966  0.216348  
R5  0.114344  0.108298  0.132491  
1     0.141677
2     0.123647
3     0.096391
4     0.131630
5     0.118101
6     0.123086
7     0.129100
8     0.135897
9     0.142763
10    0.211353
Name: AR, dtype: float64
1     0.928822
2     0.785988
3     0.579520
4     0.854301
5     0.746572
6     0.785112
7     0.834671
8     0.889535
9     0.946724
10    1.597021
Name: CR

In [48]:
# Load the monthly returns you already built
df_prices = pd.read_csv("data/monthly_returns4.csv", index_col=0)

# Build panel for, say, book-to-market deciles
panel_df, ar, cr5 = build_panel_for_char(deciles_by_year, df_prices, char_key='ep', deciles=10)

print(panel_df)  # rows R1..R5 by decile
print(ar)        # average annual return across R1..R5 per decile
print(cr5)       # 5-year compounded return per decile


           1         2         3         4         5         6         7  \
R1  0.218511  0.259526  0.219327  0.186359  0.162162  0.141284  0.138132   
R2  0.175050  0.149930  0.129714  0.152518  0.104449  0.108621  0.128862   
R3  0.178587  0.099141  0.124906  0.089504  0.137267  0.133705  0.134732   
R4  0.197867  0.096823  0.143069  0.111286  0.108970  0.099451  0.107490   
R5  0.146087  0.097596  0.164189  0.114651  0.134431  0.109840  0.107333   

           8         9        10  
R1  0.160866  0.151648  0.208449  
R2  0.135093  0.138354  0.144691  
R3  0.120605  0.125472  0.174058  
R4  0.102602  0.132027  0.136950  
R5  0.111579  0.105218  0.123138  
1     0.183220
2     0.140603
3     0.156241
4     0.130864
5     0.129456
6     0.118580
7     0.123310
8     0.126149
9     0.130544
10    0.157457
Name: AR, dtype: float64
1     1.316720
2     0.916511
3     1.062056
4     0.845260
5     0.836424
6     0.750302
7     0.787907
8     0.809778
9     0.846019
10    1.073866
Name: CR

In [53]:
def compute_weighted_sales_growth(df_statements: pd.DataFrame, min_obs=3) -> pd.DataFrame:
    """
    Lakonishok–Shleifer–Vishny (1994) GS: recency-weighted (5..1) average of
    within-year ranks of past 5Y sales growth (Y-1..Y-5), per formation year.
    Returns: columns ['gvkey','tic','gs','formation_year'].
    Firms must have >= min_obs valid past-year growth observations.
    """
    df = df_statements.copy()

    # Ensure needed cols and types
    for col in ['gvkey','tic','fyear','revt']:
        if col not in df.columns:
            raise ValueError(f"Missing column '{col}'")
    df = df.sort_values(['gvkey','fyear']).copy()
    df['fyear'] = pd.to_numeric(df['fyear'], errors='coerce').astype('Int64')

    # Sales growth per firm-year
    df['revt_lag'] = df.groupby('gvkey', group_keys=False)['revt'].shift(1)
    df['sales_growth'] = (df['revt'] / df['revt_lag']) - 1.0

    # Formation year = fyear + 1
    df['formation_year'] = df['fyear'] + 1

    rows = []

    for Y in np.sort(df['formation_year'].dropna().unique().astype(int)):
        # Past five fiscal years: Y-1..Y-5
        window_years = [Y - k for k in range(1, 6)]
        g = df.loc[df['fyear'].isin(window_years), ['gvkey','tic','fyear','sales_growth']].dropna()

        if g.empty:
            continue

        # Rank within each fiscal year; higher growth => higher rank
        g = g.copy()
        g['rank'] = g.groupby('fyear', group_keys=False)['sales_growth'].rank(method='average')
        # Normalize ranks to 0..1 per year to handle different cross-section sizes
        max_rank = g.groupby('fyear', group_keys=False)['rank'].transform('max')
        g['rank_pct'] = np.where(max_rank > 0, g['rank'] / max_rank, np.nan)

        # Recency weights: Y-1->5, Y-2->4, ..., Y-5->1
        w = 6 - (Y - g['fyear'])
        g['weight'] = w.astype(float)
        g = g.loc[g['weight'].between(1, 5)]

        if g.empty:
            continue

        # Keep firms with at least min_obs valid past-year growths
        g_valid = (g.groupby(['gvkey','tic'], group_keys=False)
                     .filter(lambda grp: grp['sales_growth'].notnull().sum() >= min_obs))

        if g_valid.empty:
            continue

        # Weighted average of rank_pct using 'weight' per firm
        # Return a 1-row Series so .reset_index() works reliably
        gs = (g_valid.groupby(['gvkey','tic'])
                      .apply(lambda x: pd.Series({
                          'gs': np.average(x['rank_pct'].to_numpy(),
                                           weights=x['weight'].to_numpy())
                      }))
                      .reset_index())

        gs['formation_year'] = Y
        rows.append(gs)

    if not rows:
        # No valid GS for any formation year
        return pd.DataFrame(columns=['gvkey','tic','gs','formation_year'])

    return pd.concat(rows, ignore_index=True)

gs_table = compute_weighted_sales_growth(df_statements)
print(gs_table.head())

/var/folders/_d/chgk8nwx2wn005yx81lw4xl80000gn/T/ipykernel_93162/389755030.py:59: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: pd.Series({
/var/folders/_d/chgk8nwx2wn005yx81lw4xl80000gn/T/ipykernel_93162/389755030.py:59: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: pd.Series({
/var/folders/_d/chgk8nwx2wn005yx81lw4xl80000gn/T/ipykernel_93162/389755030.py:59: FutureWarning: DataFrame

   gvkey     tic        gs  formation_year
0   1004     AIR  0.575769            2006
1   1072     AVX  0.384977            2006
2   1082  SERV.1  0.054370            2006
3   1111  ATVI.1  0.547543            2006
4   1173   AIM.1  0.478552            2006


/var/folders/_d/chgk8nwx2wn005yx81lw4xl80000gn/T/ipykernel_93162/389755030.py:59: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: pd.Series({


In [ ]:
def build_decile_memberships_with_gs(df_statements: pd.DataFrame,
                                     gs_table: pd.DataFrame,
                                     id_col: str = 'tic',
                                     n_bins: int = 10):
    """
    Return nested dict: deciles_by_year[char_key][year][decile] -> list of tickers.
    Uses precomputed gs_table with columns ['gvkey','tic','gs','formation_year'].
    """

    df = df_statements.copy()

    # --- Basic sanity ---
    req = {'gvkey','tic','fyear','datadate','ni','oancf','ceq','prcc_f','csho'}
    missing = req - set(df.columns)
    if missing:
        raise ValueError(f"df_statements missing columns: {missing}")

    # --- Clean & sort ---
    df['datadate'] = pd.to_datetime(df['datadate'])
    df = df.sort_values(['gvkey','datadate']).reset_index(drop=True)

    # --- Market equity & core ratios (at fiscal-year end) ---
    df['ME'] = df['prcc_f'] * df['csho']     # assumes shares in millions, price in $
    df.loc[~(df['ME'] > 0), 'ME'] = np.nan

    df['bm'] = df['ceq']   / df['ME']        # Book-to-Market
    df['ep'] = df['ni']    / df['ME']        # Earnings-to-Price
    df['cp'] = df['oancf'] / df['ME']        # CashFlow-to-Price

    # --- Formation year (end of April Y uses fyear=Y-1 financials) ---
    df['formation_year'] = pd.to_numeric(df['fyear'], errors='coerce').astype('Int64') + 1

    # --- Merge precomputed GS ---
    # Ensure types align; drop dup rows in gs_table if any
    gs = gs_table.copy()
    if 'formation_year' in gs.columns:
        gs['formation_year'] = pd.to_numeric(gs['formation_year'], errors='coerce').astype('Int64')
    gs = (gs
          .dropna(subset=['gvkey','tic','formation_year'])
          .drop_duplicates(subset=['gvkey','tic','formation_year'], keep='last'))
    # Expect columns: gvkey, tic, gs, formation_year
    if not {'gvkey','tic','gs','formation_year'}.issubset(gs.columns):
        raise ValueError("gs_table must have columns: ['gvkey','tic','gs','formation_year']")

    df = df.merge(gs[['gvkey','tic','formation_year','gs']],
                  on=['gvkey','tic','formation_year'], how='left')

    # Keep only necessary columns for decile formation
    chars = df[['formation_year','gvkey','tic','bm','ep','cp','gs','ME']].dropna(subset=['formation_year'])

    def one_char_deciles(chars_df: pd.DataFrame, char: str):
        # Use available firms with non-null characteristic
        cs = chars_df[['formation_year','gvkey','tic',char,'ME']].dropna(subset=[char]).copy()

        out_rows = []
        for Y, g in cs.groupby('formation_year', sort=True):
            # Need enough distinct values to form bins
            if g[char].nunique(dropna=True) < n_bins:
                continue

            g = g.copy()
            # Rank before qcut to avoid duplicate-edge errors
            rk = g[char].rank(method='first')
            try:
                g['decile'] = pd.qcut(rk, n_bins, labels=False) + 1  # 1..n_bins
            except ValueError:
                # still not enough spread
                continue

            members = (g.groupby('decile', sort=True)[id_col]
                         .apply(list)
                         .reindex(range(1, n_bins+1), fill_value=[])
                         .to_dict())
            out_rows.append((int(Y), members))

        # Build nested dict: deciles_by_year[year][decile] -> list
        deciles_by_year = {}
        for Y, members in out_rows:
            deciles_by_year[Y] = {int(k): v for k, v in members.items()}
        return deciles_by_year

    return {
        'bm': one_char_deciles(chars, 'bm'),
        'ep': one_char_deciles(chars, 'ep'),
        'cp': one_char_deciles(chars, 'cp'),
        'gs': one_char_deciles(chars, 'gs'),  # uses merged gs from gs_table
    }

# note that decile 1 is value for gs, but glamour for other ratios
deciles_by_year2 = build_decile_memberships_with_gs(df_statements, gs_table, id_col='tic', n_bins=10)


In [59]:
# Load the monthly returns you already built
df_prices = pd.read_csv("data/monthly_returns4.csv", index_col=0)

# Build panel for, say, book-to-market deciles
panel_df, ar, cr5 = build_panel_for_char(deciles_by_year2, df_prices, char_key='gs', deciles=10)

print(panel_df)  # rows R1..R5 by decile
print(ar)        # average annual return across R1..R5 per decile
print(cr5)       # 5-year compounded return per decile


           1         2         3         4         5         6         7  \
R1  0.183009  0.146046  0.124904  0.160756  0.123126  0.143935  0.115408   
R2  0.123007  0.120306  0.144018  0.117842  0.101011  0.125919  0.124240   
R3  0.140216  0.139340  0.149814  0.139186  0.137195  0.169998  0.134152   
R4  0.172035  0.187767  0.131890  0.141335  0.130330  0.178253  0.156298   
R5  0.157961  0.126892  0.168525  0.119723  0.182910  0.119861  0.109558   

           8         9        10  
R1  0.116074  0.096647  0.082407  
R2  0.153553  0.119221  0.105054  
R3  0.141844  0.110344  0.111434  
R4  0.152605  0.236290  0.147169  
R5  0.129683  0.166986  0.127041  
1     0.155246
2     0.144070
3     0.143830
4     0.135768
5     0.134914
6     0.147593
7     0.127931
8     0.138752
9     0.145898
10    0.114621
Name: AR, dtype: float64
1     1.055854
2     0.957968
3     0.957125
4     0.889032
5     0.880234
6     0.988367
7     0.824667
8     0.914144
9     0.966195
10    0.718798
Name: CR